# モデルがFitしているケース

In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression

def sigmoid(self, x):
    return 1./(1+np.exp(-x))

class simulater():
    def set_num_data(self, num_data):
        self.num_data = num_data

    def generate_linear_data(self):
        num_data = self.num_data

        self.df = pd.DataFrame({
            "x1": np.random.normal(size=num_data),
            "x2": np.random.normal(size=num_data),    
        }).assign(
            prob_t = lambda df: sigmoid(df.x1**2+df.x2),
            t = lambda df: (df.prob_t >= np.random.uniform(size=num_data))*1,
            reward = lambda df: df.x1 - df.x2 + df.t * 2
        )
        return self.df

    def generate_nonlinear_data(self):
        num_data = self.num_data

        self.df = pd.DataFrame({
            "x1": np.random.normal(size=num_data),
            "x2": np.random.normal(size=num_data),    
        }).assign(
            prob_t = lambda df: sigmoid(df.x1**2+df.x2),
            t = lambda df: (df.prob_t >= np.random.uniform(size=num_data))*1,
            reward = lambda df: df.x1**4 - df.x2**4 + df.t
        )
        return self.df

    def IPS(self):
        df = self.df
        mode = LogisticRegression().fit(X=df[["x1", "x2"]], y=df["t"]==1)
        df["t_pred"] = mode.predict_proba(df[["x1", "x2"]])[:,1]

        df["reward_treatment"] = df["reward"]*df["t"]/df["t_pred"]
        df["reward_no_treatment"] = df["reward"]*(1-df["t"])/(1-df["t_pred"])

        return abs((df["reward_treatment"]-df["reward_no_treatment"]).mean()-1)

    def DM(self):
        df = self.df
        mode = LinearRegression().fit(X=df[["x1", "x2", "t"]], y=df["reward"])
        df["reward_treatment"] = mode.predict(df[["x1", "x2", "t"]].assign(t=1))
        df["reward_no_treatment"] = mode.predict(df[["x1", "x2", "t"]].assign(t=0))

        return abs((df["reward_treatment"]-df["reward_no_treatment"]).mean()-1)
    
    def sim_linear(self):
        ress= []
        for num_data in [100, 200, 400, 800, 1600, 3200]:
            self.generate_linear_data()
            res = {"num_data":num_data}
            res["IPS"] = self.IPS()
            res["DM"] = self.DM()
            ress.append(res)
        return pd.DataFrame(ress)

    def sim_nonlinear(self):
        ress= []
        for num_data in [100, 200, 400, 800, 1600, 3200]:
            self.generate_linear_data()
            res = {"num_data":num_data}
            res["IPS"] = self.IPS()
            res["DM"] = self.DM()
            ress.append(res)
        return pd.DataFrame(ress)

# IPS / DM

1.830871230517721
2.0
